In [1]:
import numpy as np
from functools import reduce
import scipy.sparse as sps
import scipy.sparse.linalg as spslin
import sys
np.set_printoptions(threshold=sys.maxsize)


In [2]:
# Loading the Pauli matrices

Id=sps.eye(2)
Sx=sps.csr_matrix(np.array([[0,1.],[1.,0]]))
Sy=sps.csr_matrix(np.array([[0,-1.j],[1.j,0]]))
Sz=sps.csr_matrix(np.array([[1.,0.],[0,-1.]]))

In [3]:
## Loading the adjacency list

Lx,Ly=2,2
N_2d=2*Lx*Ly
s=np.arange(N_2d)
T1=np.array([3,2,4,5,7,6,0,1])
T2=np.array([2,3,5,4,6,7,1,0])
I1=np.array([6,7,1,0,2,3,5,4])
I2=np.array([7,6,0,1,3,2,4,5])
Tx=np.array([1,0,3,2,5,4,7,6])

In [4]:
J_tc=1.
As=[[-J_tc,i,Tx[i],T1[i],I2[i]] for i in [0,1,4,5]]
Bp=[[-J_tc,i,Tx[i],T1[i],I2[i]] for i in [2,3,6,7]]

static=[["xxxx",As],["zzzz",Bp]]
static

[['xxxx',
  [[-1.0, 0, 1, 3, 7],
   [-1.0, 1, 0, 2, 6],
   [-1.0, 4, 5, 7, 3],
   [-1.0, 5, 4, 6, 2]]],
 ['zzzz',
  [[-1.0, 2, 3, 4, 0],
   [-1.0, 3, 2, 5, 1],
   [-1.0, 6, 7, 0, 4],
   [-1.0, 7, 6, 1, 5]]]]

In [5]:
def ham_general(L,static):
    ham_gen=np.zeros(shape=(2**L,2**L),dtype=float)
    for item in static:
        string=item[0]
        for inside in item[1:len(item)]:
            for j in range(len(inside)):
                op_list=[Id]*L
                for k in range(len(string)):
                    if string[k]=='x':
                        op_list[inside[j][k+1]]=Sx
                    elif string[k]=='y':
                        op_list[inside[j][k+1]]=Sy
                    elif string[k]=='z':
                        op_list[inside[j][k+1]]=Sz
                #print(reduce(sps.kron,op_list).toarray())
                ham_gen += inside[j][0]*reduce(sps.kron,op_list).toarray()
    #            
    return ham_gen

In [6]:
H=ham_general(N_2d,static)
print(H.shape)
(e2,u) = spslin.eigsh(H, k=255, which='SA',return_eigenvectors=True)

e2

(256, 256)


array([-8.00000000e+00, -8.00000000e+00, -8.00000000e+00, -8.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -4.00000000e+00, -4.00000000e+00, -4.00000000e+00, -4.00000000e+00,
       -1.01588358e-14, -

In [7]:
# with wilson loop

J_tcw=1.
As_w=[[-J_tcw,i,Tx[i],T1[i],I2[i]] for i in [0,1,4,5]]
Bp_w=[[-J_tcw,i,Tx[i],T1[i],I2[i]] for i in [2,3,6,7]]

gz_w=0.1
Wx=[[-gz_w,0,1]]
Wy=[[-gz_w,2,6]]

static_w=[["xxxx",As_w],["zzzz",Bp_w],["zz",Wx],["zz",Wy]]

In [8]:
(e2w,uw) = spslin.eigsh(ham_general(N_2d,static_w), k=16, which='SA',return_eigenvectors=True)

e2w

array([-8.2, -8. , -8. , -7.8, -4.2, -4.2, -4.2, -4.2, -4.2, -4.2, -4.2,
       -4.2, -4.2, -4.2, -4.2, -4.2])